# Getting Drake's top tracks along with albums

In [52]:
import requests
import pandas as pd
import base64
import datetime
from urllib.parse import urlencode

In [53]:
client_id='445e1a6b603d4e93a0485e1a71fd1548'
secret_key='145654bd9a094c2398f7f2b6850a05ed'

base_url = 'https://api.spotify.com/v1/'
endpoint = ''
token_url = 'https://accounts.spotify.com/api/token'
request_type ='POST'


## Creating all necessary functions


In [71]:
class SpotifyApi(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    secret_key = None
    token_url = 'https://accounts.spotify.com/api/token'
    
    def __init__(self, client_id, secret_key, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.secret_key = secret_key
        
 #Getting everything encoded and ready   
    def get_client_credentials(self):
        client_id = self.client_id
        secret_key = self.secret_key
        client_creds = f'{client_id}:{secret_key}'
        client_creds_b64 = base64.b64encode(client_creds.encode()).decode()
        return client_creds_b64       
          
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {'Authorization' : f'Basic {client_creds_b64}'}
    
    def get_token_data(self):
        return {
            'grant_type':'client_credentials'
        }
    
 #Performing Oauth's open protocol secure authorization   
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r=requests.post(token_url, data=token_data, headers=token_headers)
        valid_request = r.status_code in range(200, 299)
        if not valid_request:
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
 #Getting the required access token   
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
  # Searching your favourite albums , artists, playlists, tracks, shows.  
    def search(self, query, search_type='arist'):
        access_token = self.get_access_token()       
        headers = {
            'Authorization' : f'Bearer {access_token}'
        }
        endpoint = 'search' 
        data = urlencode({'q': query,'type':search_type.lower()})
        lookup_url = f'{base_url}{endpoint}?{data}'
        print(lookup_url)
        r = requests.get(lookup_url, headers= headers)
        if r.status_code not in range(200, 299):
            return{}
        return r.json()
    
    #Getting an artist's top tracks
    #https://api.spotify.com/v1/artists/{id}/top-tracks
    def artist_top_tracks(self, id_):
        access_token = self.get_access_token()       
        headers = {
            'Authorization' : f'Bearer {access_token}'
        }
        endpoint = f'artists/{id_}/top-tracks?market=US'
        lookup_url = f'{base_url}{endpoint}'
        print(lookup_url)
        r = requests.get(lookup_url, headers= headers)
        if r.status_code not in range(200, 299):
            return{}
        return r.json()
        
        

## Getting JSON data

In [72]:
spotify = SpotifyApi(client_id, secret_key)

In [64]:
spotify.search('Drake', search_type='artist')

https://api.spotify.com/v1/search?q=Drake&type=artist


{'artists': {'href': 'https://api.spotify.com/v1/search?query=Drake&type=artist&offset=0&limit=20',
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'},
    'followers': {'href': None, 'total': 56431559},
    'genres': ['canadian hip hop',
     'canadian pop',
     'hip hop',
     'pop rap',
     'rap',
     'toronto rap'],
    'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4',
    'id': '3TVXtAsR1Inumwj472S9r4',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab6761610000e5eb9d6c72d285ab349ce7a93529',
      'width': 640},
     {'height': 320,
      'url': 'https://i.scdn.co/image/ab676161000051749d6c72d285ab349ce7a93529',
      'width': 320},
     {'height': 160,
      'url': 'https://i.scdn.co/image/ab6761610000f1789d6c72d285ab349ce7a93529',
      'width': 160}],
    'name': 'Drake',
    'popularity': 98,
    'type': 'artist',
    'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'},
   {'external_urls'

In [77]:
#Getting Drake's top hits
spotify_data = spotify.artist_top_tracks(id_='3TVXtAsR1Inumwj472S9r4')

https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4/top-tracks?market=US


In [109]:
#Creating empty lists to store the data
album_name_list1 = []
release_date_list1 = []
tracks_number_list1 = []
track_name_list1 = []
popularity_number_list1 = []





## Extracting Data from JSON 


In [110]:
for i in range(0, len(spotify_data['tracks'])):
    album_name_list1.append(spotify_data['tracks'][i]['album']['name'])
    release_date_list1.append(spotify_data['tracks'][i]['album']['release_date'])
    tracks_number_list1.append(spotify_data['tracks'][i]['album']['total_tracks'])
    track_name_list1.append(spotify_data['tracks'][i]['name'])
    popularity_number_list1.append(spotify_data['tracks'][i]['popularity'])
    

## Creating Table and generating csv file


In [113]:
table = pd.DataFrame(
        { 'Track': track_name_list1,
          'Popularity': popularity_number_list1,
          'Album': album_name_list1,
          'Release Date': release_date_list1,
          'No. of Tracks': tracks_number_list1,
        
        }
)

In [114]:
table.head()

,Track,Popularity,Album,Release Date,No. of Tracks
0,Wants and Needs (feat. Lil Baby),87,Scary Hours 2,2021-03-05,3
1,One Dance,83,Views,2016-05-06,20
2,Over The Top (feat. Drake),83,Over The Top (feat. Drake),2021-07-23,1
3,Laugh Now Cry Later (feat. Lil Durk),83,Laugh Now Cry Later (feat. Lil Durk),2020-08-14,1
4,God's Plan,82,Scorpion,2018-06-29,25


In [116]:
table.to_csv('Drake popular tracks.csv', index=False)